- Link Model: https://huggingface.co/pitangent-ds/YOLOv8-human-detection-thermal
- Link Dataset: https://universe.roboflow.com/smart2/persondection-61bc2/dataset/5#

In [ ]:
# Cài đặt các thư viện cần thiết cho YOLOv8
!pip install ultralytics supervision huggingface_hub

# 1. Data

Link: https://universe.roboflow.com/smart2/persondection-61bc2/dataset/
Chọn YoloV8 - Show download code

In [ ]:
!pip install roboflow

from roboflow import Roboflow

# Nhập API key của bạn
rf = Roboflow(api_key="R1Rsh0Bx3eiHdB9aSGr2")  # Thay thế bằng API key của bạn

# Kết nối đến workspace và project trên Roboflow
project = rf.workspace("smart2").project("persondection-61bc2")

# Chọn phiên bản project để tải dữ liệu
version = project.version(5)

# Tải dataset dưới định dạng YOLOv8
dataset = version.download("yolov8")



Vẽ cấu trúc thư mục sau khi down về bằng markdown
```bash
/kaggle/working/PersonDection-5/
│
├── valid/
│   ├── images/
│   └── labels/
│
├── train/
│   ├── images/
│   └── labels/
│
├── test/
│   ├── images/
│   └── labels/
│
├── data.yaml
├── README.dataset.txt
├── README.roboflow.txt
└── roboflow.zip
```

Kiểm tra dataset: 8000 imgs train, 1000 valid, 1000 test

# 2. Test Model Trước Khi Huấn Luyện (Inference)

## 2.1 Test 1 ảnh

In [ ]:
# Import các thư viện
from huggingface_hub import hf_hub_download
from ultralytics import YOLO
from supervision import Detections
import cv2 as cv

# Tải mô hình YOLOv8 đã được huấn luyện
model_path = hf_hub_download(
    repo_id="pitangent-ds/YOLOv8-human-detection-thermal",
    filename="model.pt"
)

# Khởi tạo mô hình từ file .pt đã tải
model = YOLO(model_path)


In [ ]:
# Hàm để chạy suy luận
def inference(image_path):
    # Đọc ảnh từ đường dẫn
    cv_image = cv.imread(image_path, cv.IMREAD_ANYCOLOR)
    
    # Chạy mô hình YOLO trên ảnh
    model_output = model(cv_image, conf=0.6, verbose=False)
    
    # Xử lý kết quả đầu ra
    detections = Detections.from_ultralytics(model_output[0])
    
    # Trả về kết quả suy luận
    return detections

# Chạy suy luận trên ảnh cụ thể
detections = inference('/kaggle/working/PersonDection-5/test/images/video-4FRnNpmSmwktFJKjg-frame-001044-YYpRCPPF5hNtz2bkA_jpg.rf.a1fcf98068d16a6ffd22e9e6d4fbd1e6.jpg')
print(detections)


- xyxy: Tọa độ của bounding box (477.36, 258.71, 505.37, 318.28). Đây là các tọa độ điểm giới hạn của hộp bao quanh đối tượng trong ảnh. 
    - x1, y1: Tọa độ của góc trên bên trái của hộp bao quanh (bounding box).
    - x2, y2: Tọa độ của góc dưới bên phải của hộp bao quanh.
    - Trong YOLO, gốc tọa độ (0, 0) thường nằm ở góc trên bên trái của hình ảnh, trục x nằm ngang từ trái sang phải và trục y nằm từ trên xuống dưới.
    - Trong hệ toạ độ Oxy Descartes, gốc tọa độ (0, 0) thường nằm ở giữa hệ trục, và trục y chạy từ dưới lên trên, trái ngược với YOLO.
- confidence: Độ tự tin của mô hình là 0.75238 (tức là 75.238%), cho thấy mức độ chắc chắn của mô hình về việc đối tượng được phát hiện là con người.
- class_id: 0 (tương ứng với lớp "HUMAN" trong mô hình của bạn).
- class_name: Tên lớp của đối tượng là HUMAN.


In [ ]:
import cv2
import matplotlib.pyplot as plt

# Đọc ảnh
image = cv2.imread('/kaggle/working/PersonDection-5/test/images/video-4FRnNpmSmwktFJKjg-frame-001044-YYpRCPPF5hNtz2bkA_jpg.rf.a1fcf98068d16a6ffd22e9e6d4fbd1e6.jpg')

# Kiểm tra nếu ảnh được đọc thành công
if image is not None:
    # Lấy tọa độ bounding box
    x1, y1, x2, y2 = int(477.36), int(258.71), int(505.37), int(318.28)

    # Vẽ bounding box trên ảnh
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Màu xanh lá với độ dày 2px

    # Ghi nhãn đối tượng
    cv2.putText(image, "HUMAN", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Chuyển đổi từ BGR sang RGB để hiển thị với matplotlib
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Hiển thị ảnh với matplotlib
    plt.imshow(image_rgb)
    plt.axis('off')  # Ẩn trục tọa độ
    plt.show()
else:
    print("Không thể đọc ảnh, vui lòng kiểm tra lại đường dẫn.")


# 3. Đánh giá model trước, trong, sau huấn luyện

## 3.1 Đánh giá mô hình trước khi huấn luyện (trên tập validation):

In [ ]:
from ultralytics import YOLO

# Tải mô hình YOLOv8n đã được huấn luyện sẵn
model = YOLO("yolov8n.pt")

# Chạy đánh giá trên tập validation
metrics_val = model.val(data='/kaggle/working/PersonDection-5/data.yaml')  # Sử dụng tập validation

# In các chỉ số đánh giá trước khi huấn luyện
print("Đánh giá trước khi huấn luyện trên tập validation:")
print(f"Precision: {metrics_val.box.map50:.3f}")
print(f"Recall: {metrics_val.box.map:.3f}")
print(f"mAP@0.5: {metrics_val.box.map50:.3f}")
print(f"mAP@0.5:0.95: {metrics_val.box.map:.3f}")


- Precision 0.526 và mAP@0.5 0.526 là mức độ khá trung bình, nhưng điều này chỉ là trước khi huấn luyện, có nghĩa là mô hình chưa được tối ưu và chỉ dựa trên mô hình YOLOv8 cơ bản.
- Recall 0.333 khá thấp, chỉ ra rằng mô hình hiện tại không phát hiện được nhiều đối tượng trong dữ liệu. Điều này rất phổ biến trước khi huấn luyện vì mô hình chưa học được nhiều thông tin từ dữ liệu cụ thể.
- mAP@0.5:0.95 0.333: Đây là một chỉ số quan trọng thể hiện mức độ tổng quát hóa của mô hình khi các yêu cầu về độ chồng lấn (IoU) càng khắt khe hơn. Với chỉ số thấp, mô hình trước huấn luyện khó có khả năng dự đoán chính xác khi các đối tượng trong ảnh bị chồng chéo phức tạp.

## 3.2 Huấn luyện mô hình trên tập training và validation:Huấn luyện mô hình trên tập training và validation:

### Bảng tóm tắt về thời gian huấn luyện và các phương pháp tăng tốc:

| **Yếu tố**                | **Dự tính/Khuyến nghị**                                                               |
|---------------------------|----------------------------------------------------------------------------------------|
| **Số lượng ảnh**           | 8000 ảnh huấn luyện, 1000 ảnh xác thực.                                                |
| **Thời gian huấn luyện**   | 50-250 phút (khoảng 1 đến 4 giờ) trên GPU Tesla T4 (50 epochs, imgsz=640x640).        |
| **Sử dụng mô hình nhẹ hơn**| Dùng YOLOv8n (nano) để giảm thời gian huấn luyện, nhưng có thể giảm độ chính xác.      |
| **Giảm kích thước ảnh**    | Giảm kích thước ảnh từ 640x640 xuống 416x416 để tăng tốc độ huấn luyện.               |
| **Giảm số lượng epochs**   | Thử huấn luyện với 30 epochs để rút ngắn thời gian và theo dõi kết quả sau đó.        |
| **Mixed Precision Training**| Sử dụng huấn luyện với precision hỗn hợp (`half=True`) để tăng tốc độ huấn luyện.     |


Mình chọn: log vào MLflow vì mình chưa dùng bao giờ (W&B mình từng dùng rùi)

In [ ]:
!pip install wandb
!pip install -U ipywidgets # cập nhật từ 7.7.1 lên 8.0 

In [ ]:
import os
import wandb

# # Need Enter key
# wandb.login()

# # Đăng nhập bằng biến môi trường
# wandb.login(key=os.getenv("WANDB_API_KEY"))

# Đăng nhập trực tiếp vào WandB bằng API key
wandb.login(key="c8767797aae76cbcd389ff29929ace1ac3021161")

In [ ]:
# # Tải mô hình YOLOv8n
# model = YOLO("yolov8n.pt")

# # Huấn luyện mô hình và tích hợp WandB với tên chi tiết
# model.train(
#     data='/kaggle/working/PersonDection-5/data.yaml',  # Đường dẫn tới file cấu hình dữ liệu
#     epochs=3,                   # Số lượng epochs
#     imgsz=640,                   # Kích thước ảnh
#     project="ThermalHumanDetect_YOLO",     # Tên project trên WandB
#     name="YOLOv8n_ThermalHumanDetect_img640_epochs3_mixedPrecision_noAug_lr0.01",  # name RUN, Tên chi tiết
#     save_period=1,               # Lưu kết quả mỗi 1 epochs
#     device="cuda",               # Sử dụng GPU
#     half=True,                   # Mixed precision để tăng tốc độ huấn luyện
#     lr0=0.01                     # Learning rate ban đầu là 0.01
# )


In [ ]:
# # Đánh giá mô hình trên tập test
# metrics_test = model.val(data='/kaggle/working/PersonDection-5/data.yaml', split="test")

# # In các chỉ số sau huấn luyện
# print("Kết quả đánh giá trên tập test sau huấn luyện:")
# print(f"Precision: {metrics_test.box.map50:.3f}")
# print(f"Recall: {metrics_test.box.map:.3f}")
# print(f"mAP@0.5: {metrics_test.box.map50:.3f}")
# print(f"mAP@0.5:0.95: {metrics_test.box.map:.3f}")


In [ ]:
# # Tải mô hình YOLOv8n
# model = YOLO("yolov8n.pt")

# # Huấn luyện mô hình và tích hợp WandB với tên chi tiết
# model.train(
#     data='/kaggle/working/PersonDection-5/data.yaml',  # Đường dẫn tới file cấu hình dữ liệu
#     epochs=15,                   # Số lượng epochs
#     imgsz=640,                   # Kích thước ảnh
#     project="ThermalHumanDetect_YOLO",     # Tên project trên WandB
#     name="YOLOv8n_ThermalHumanDetect_img640_epochs15_mixedPrecision_noAug_lr0.01",  # name RUN, Tên chi tiết
#     save_period=1,               # Lưu kết quả mỗi 1 epochs
#     device="cuda",               # Sử dụng GPU
#     half=True,                   # Mixed precision để tăng tốc độ huấn luyện
#     lr0=0.01                     # Learning rate ban đầu là 0.01
# )


Tóm tắt các lựa chọn mô hình YOLOv8 phù hợp cho **8000 ảnh train**, **1000 ảnh validation** và GPU **T4 x2** trên Kaggle:

| **Phiên bản YOLOv8** | **Ưu tiên**                      | **Độ chính xác**                 | **Thời gian dự kiến (50 epochs)** | **Khuyến nghị**                                                                                 |
|-----------------------|----------------------------------|-----------------------------------|-----------------------------------|-------------------------------------------------------------------------------------------------|
| **YOLOv8n (Nano)**    | Tốc độ tối đa, tài nguyên nhẹ   | Thấp nhất                         | < 1 giờ                           | Phù hợp cho các thử nghiệm nhanh hoặc các ứng dụng cần tốc độ cao, chấp nhận độ chính xác thấp. |
| **YOLOv8s (Small)**   | Cân bằng giữa tốc độ và độ chính xác | Tốt, độ chính xác vừa phải         | 1-2 giờ                           | Lựa chọn tốt nếu cần cân bằng giữa hiệu suất và thời gian huấn luyện.                           |
| **YOLOv8m (Medium)**  | Độ chính xác cao hơn YOLOv8s    | Cao                               | 2-4 giờ                           | Phù hợp cho các bài toán yêu cầu độ chính xác cao hơn và sẵn sàng đợi lâu hơn để huấn luyện.    |

### Gợi ý
- **YOLOv8s** là lựa chọn **cân bằng** giữa thời gian và độ chính xác cho bài toán của bạn.
- **YOLOv8m** nếu bạn cần **độ chính xác cao hơn** và có thể dành nhiều thời gian cho quá trình huấn luyện.

In [ ]:
# # Đánh giá mô hình trên tập test
# metrics_test = model.val(data='/kaggle/working/PersonDection-5/data.yaml', split="test")

# # In các chỉ số sau huấn luyện
# print("Kết quả đánh giá trên tập test sau huấn luyện:")
# print(f"Precision: {metrics_test.box.map50:.3f}")
# print(f"Recall: {metrics_test.box.map:.3f}")
# print(f"mAP@0.5: {metrics_test.box.map50:.3f}")
# print(f"mAP@0.5:0.95: {metrics_test.box.map:.3f}")


| **Chỉ Số**        | **Ý Nghĩa**                                                                                          | **Thế nào là tốt?**                                                                                     | **Validation - trước huấn luyện** | **Validation - 3 epochs - Yolov8s - No Augmentation** | **Validation - 15 epochs - Yolov8s - No Augmentation** | **Test - 15 epochs - Yolov8s - No Augmentation** |
|-------------------|------------------------------------------------------------------------------------------------------|---------------------------------------------------------------------------------------------------------|-------------------------------|--------------------------|--------------------------|--------------------|
| **Time Training/Inference** | Thời gian huấn luyện hoặc suy luận cho mỗi lần đánh giá mô hình. | Thời gian ngắn giúp tối ưu hiệu suất mô hình. | N/A | 0.08 giờ | 0.4 giờ | 0.01 giờ |
| **Precision**      | Tỉ lệ giữa số dự đoán đúng (true positives) và tổng số các dự đoán (true positives + false positives). Precision cao nghĩa là mô hình ít dự đoán sai. | Precision cao thể hiện độ chính xác của mô hình trong việc đưa ra dự đoán đúng.                          | 0.526                        | 0.808                   | 0.865                   | 0.904             |
| **Recall**         | Tỉ lệ giữa số dự đoán đúng và tổng số đối tượng thực sự có (true positives + false negatives). Recall cao nghĩa là mô hình phát hiện tốt các đối tượng. | Recall cao thể hiện khả năng phát hiện đầy đủ các đối tượng trong dữ liệu.                               | 0.333                        | 0.713                   | 0.811                   | 0.597             |
| **mAP@0.5**        | Mean Average Precision tại ngưỡng IoU = 0.5, đo lường độ chính xác của dự đoán khi yêu cầu độ chồng lấn giữa các hộp giới hạn là 50%. | mAP@0.5 cao cho thấy mô hình có khả năng phát hiện tốt các đối tượng với độ chính xác hộp giới hạn (bounding boxes). | 0.526                        | 0.808                   | 0.898                   | 0.904             |
| **mAP@0.5:0.95**   | Mean Average Precision trung bình qua nhiều ngưỡng IoU từ 0.5 đến 0.95. Chỉ số này phản ánh khả năng dự đoán chính xác với các mức độ chồng lấn khác nhau. | mAP@0.5:0.95 cao thể hiện khả năng tổng quát hóa tốt của mô hình, đặc biệt trong các bài toán có các đối tượng chồng lấn. | 0.333                        | 0.499                   | 0.618                   | 0.597             |



# 4. Improve Model - Yolov8s - 30 epochs - withAugmentation

In [ ]:
from ultralytics import YOLO

# Tải mô hình YOLOv8m
model = YOLO("yolov8s.pt")

# Huấn luyện mô hình và tích hợp WandB với tên chi tiết
model.train(
    data='/kaggle/working/PersonDection-5/data.yaml',  # Đường dẫn tới file cấu hình dữ liệu
    epochs=30,                   # Số lượng epochs
    imgsz=640,                   # Kích thước ảnh
    project="ThermalHumanDetect_YOLO",     # Tên project trên WandB
    name="YOLOv8s_ThermalHumanDetect_img640_epochs30_mixedPrecision_withAug_lr0.01",  # name RUN, Tên chi tiết
    save_period=10,               # Lưu kết quả mỗi 10 epochs - CHỈ LƯU Ở KAGGLE, KO LƯU TRÊN WANDB ĐƯỢC. 
    device="cuda",               # Sử dụng GPU
    half=True,                   # Mixed precision để tăng tốc độ huấn luyện
    lr0=0.01,                    # Learning rate ban đầu là 0.01
    augment=True                 # Kích hoạt augmentation
)


In [ ]:
# Đánh giá mô hình trên tập test
metrics_test = model.val(data='/kaggle/working/PersonDection-5/data.yaml', split="test")

# In các chỉ số sau huấn luyện
print("Kết quả đánh giá trên tập test sau huấn luyện:")
print(f"Precision: {metrics_test.box.map50:.3f}")
print(f"Recall: {metrics_test.box.map:.3f}")
print(f"mAP@0.5: {metrics_test.box.map50:.3f}")
print(f"mAP@0.5:0.95: {metrics_test.box.map:.3f}")


Finish